## Importing data and pre-processing

In [ ]:
pip install pycaret -q

In [ ]:
# Importing relevant packages

import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import make_pipeline, Pipeline

from pycaret.regression import *

%matplotlib inline

In [ ]:
# Importing data
hitters = pd.read_excel('/content/MLBFreeAgentData.xlsx')

In [ ]:
# Displaying data
hitters

,Year,Player,Pos'n,Age,Old Club,New Club,Years,Guarantee,Term,Player Agent,Club Owner,Baseball Ops head / club GM,Future WAR,past WAR,HR,RBI,IBB,SB,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,RAR,Spd,wRC+,WPA,RE24,REW,Clutch
0,1999,"Alomar, Roberto",2b,31,BAL,CLE,4,32000000,1999-2002,Jaime Torres,Richard Jacobs,John Hart,20.3,7.3,28,116,5,27,0.303,0.088,0.100,0.88,0.365,0.452,0.817,0.149,0.316,0.358,24.6,164,74.8,4.5,115,-0.44,8.42,0.92,-1.62
1,1999,"Palmeiro, Rafael",1b,34,BAL,TEX,5,45000000,1999-2003,Bronner Gilhooley,Tom Hicks,Doug Melvin,20.0,10.2,81,231,15,16,0.275,0.104,0.143,0.73,0.355,0.526,0.880,0.251,0.268,0.376,51.4,224,103.3,3.4,127,4.45,47.41,4.57,0.28
2,1999,"Ventura, Robin",3b,31,CHA,NYN,4,32000000,1999-2002,John Boggs,Wilpon/Doubleday,Steve Phillips,15.7,7.6,27,117,20,1,0.263,0.126,0.148,0.86,0.355,0.433,0.788,0.171,0.284,0.341,6.9,117,77.0,3.0,104,3.06,11.46,1.05,3.00
3,1999,"Burks, Ellis",of,34,SFN,SFN,2,10000000,1999-2000,Jim Turner,Peter Magowan,Brian Sabean,7.3,3.3,53,158,1,18,0.291,0.099,0.175,0.56,0.364,0.530,0.894,0.239,0.310,0.385,46.6,177,33.7,5.1,114,1.56,33.07,3.30,-1.83
4,1999,"Finley, Steve",of,34,SDN,ARI,5,21500000,1999-2003,Tommy Tanzer,Jerry Colangelo,Joe Garagiola Jr.,17.0,0.8,42,159,2,27,0.254,0.068,0.151,0.45,0.307,0.436,0.743,0.182,0.271,0.323,-10.0,149,8.8,6.3,97,0.31,7.01,0.54,-0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,2019,"Kang, Jung Ho",3b,32,PIT,PIT,1,3000000,2019,Wasserman,Bob Nutting,Neal Huntington,-0.9,0.0,0,0,0,0,0.333,0.000,0.167,0.00,0.333,0.333,0.667,0.000,0.400,0.293,-0.1,1,-0.1,0.1,83,-0.01,0.11,0.01,0.03
880,2019,"Maldonado, Martin",c,32,HOU,KCA,1,2500000,2019,Magnus Sports,David Glass,Dayton Moore,0.8,5.0,23,82,1,0,0.223,0.035,0.248,0.14,0.276,0.360,0.637,0.137,0.276,0.277,-29.8,75,49.3,2.4,72,-5.34,-36.78,-3.85,-2.55
881,2019,"McCann, Brian",c,35,HOU,ATL,1,2000000,2019,Jet Sports,Liberty Media,Alex Anthopoulos,1.1,2.7,25,85,3,1,0.230,0.093,0.159,0.58,0.315,0.401,0.717,0.171,0.235,0.310,-4.5,69,27.5,2.3,96,0.14,-9.64,-0.94,-0.04
882,2019,"Harrison, Josh",2b-3b,31,PIT,DET,1,2000000,2019,MSM Sports,Chris Ilitch,Al Avila,-0.7,2.9,24,84,3,15,0.263,0.050,0.172,0.29,0.320,0.404,0.724,0.141,0.296,0.313,-4.1,106,29.2,4.7,93,-0.84,-1.62,-0.14,-0.21


In [ ]:
# Looking at information of data
hitters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year                         884 non-null    int64  
 1   Player                       884 non-null    object 
 2   Pos'n                        884 non-null    object 
 3   Age                          884 non-null    int64  
 4   Old Club                     882 non-null    object 
 5   New Club                     884 non-null    object 
 6   Years                        884 non-null    int64  
 7   Guarantee                    884 non-null    int64  
 8   Term                         884 non-null    object 
 9   Player Agent                 768 non-null    object 
 10  Club Owner                   882 non-null    object 
 11  Baseball Ops head / club GM  882 non-null    object 
 12  Future WAR                   884 non-null    float64
 13  past WAR            

In [ ]:
# Changing the name of position column to avoid python issues

hitters.rename(columns={'Pos\'n':'Position'}, inplace=True)

# Filling in null values of the dataframe

for column in hitters.select_dtypes('object'):
  hitters[column].fillna('unknown', inplace=True)

hitters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year                         884 non-null    int64  
 1   Player                       884 non-null    object 
 2   Position                     884 non-null    object 
 3   Age                          884 non-null    int64  
 4   Old Club                     884 non-null    object 
 5   New Club                     884 non-null    object 
 6   Years                        884 non-null    int64  
 7   Guarantee                    884 non-null    int64  
 8   Term                         884 non-null    object 
 9   Player Agent                 884 non-null    object 
 10  Club Owner                   884 non-null    object 
 11  Baseball Ops head / club GM  884 non-null    object 
 12  Future WAR                   884 non-null    float64
 13  past WAR            

In [ ]:
# Checking the number of unique players in the dataframe
print(hitters['Player'].nunique())

492


## Exploratory Data Analysis

In [ ]:
#@title Creating a filter to filter the data by year, to observe changes over time.
#1999-2014 is defined as pre-Moneyball
#2005-2010 is defined as "post-Moneyball"
start = "1999" #@param [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
end = "2019" #@param [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

start = int(start)
end = int(end)

hitters = hitters[hitters['Year'] >= start]
hitters = hitters[hitters['Year'] <= end]


In [ ]:
# Checking years for the analysis

hitters['Year'].unique()

array([1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007])

In [ ]:
hitters.columns

Index(['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years',
       'Guarantee', 'Term', 'Player Agent', 'Club Owner',
       'Baseball Ops head / club GM', 'Future WAR', 'past WAR', 'HR', 'RBI',
       'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO',
       'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+', 'WPA', 'RE24',
       'REW', 'Clutch'],
      dtype='object')

In [ ]:
#@title Histograms

variable = 'Guarantee' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']
split_by = 'Year' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']

# Histogram of contracts

px.histogram(hitters.sort_values(by=split_by),
             x= variable,
             marginal='box',
             color=split_by)

In [ ]:
#@title Mean Contract Value

# Checking Players / Agents / Club Owners associated with high value contracts

variable = 'Guarantee' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']
split_by = "Position" #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']

# Getting mean value of contrat amount

hitters_mean = hitters.groupby([split_by])[variable].mean().reset_index()
hitters_mean.sort_values(by=variable, ascending=True, inplace=True)

# Exporting data to a csv file

hitters_mean.to_csv('hitters_mean.csv')

# Bar plots

px.bar(hitters_mean,
       x= split_by,
       y= variable,
      #  hover_data=['Player', 'Year', 'Guarantee', 'Years']
       )

In [ ]:
variable = 'Guarantee' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']
split_by = "Position" #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']

# hitters.sort_values(by=variable, ascending=True, inplace=True)

fig = px.box(hitters,
       x= split_by,
       y= variable,
       hover_data=['Player', 'Year', 'Guarantee', 'Position']
       )

fig.show()

# hitters.sort_values(by='Year', ascending=True, inplace=True)

In [ ]:
#Analyzing the correlations of performance statistics to free agents pay (Guarantee). Guarantee can be replaced with 'Future WAR' to analyze the correlations of the variables with future performance.
correlation_years = pd.DataFrame()

for year in hitters['Year'].unique():
  hitters_corr = pd.DataFrame(hitters[hitters['Year'] == year].corr())
  hitters_corr = pd.DataFrame(hitters_corr['Guarantee'])
  correlation_years[year] = hitters_corr['Guarantee']

correlation_years = correlation_years.transpose()

correlation_years

,Year,Age,Years,Guarantee,Future WAR,past WAR,HR,RBI,IBB,SB,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,RAR,Spd,wRC+,WPA,RE24,REW,Clutch
1999,NaN,-0.342099,0.915509,1.0,0.626074,0.809928,0.637246,0.699337,0.636346,0.073323,0.507399,0.187479,-0.051743,0.157884,0.463601,0.649828,0.669933,0.534503,0.281107,0.634415,0.749821,0.738878,0.811750,0.079444,0.630891,0.545204,0.646741,0.637458,-0.254659
2000,NaN,-0.199545,0.838472,1.0,0.536491,0.748215,0.824683,0.798337,0.462568,0.192755,0.042881,0.215936,0.128887,0.174399,0.202986,0.433704,0.414757,0.493516,-0.230846,0.388584,0.636070,0.769393,0.750420,0.072672,0.476392,0.642554,0.645914,0.664716,-0.159722
2001,NaN,-0.562004,0.914147,1.0,0.972026,0.780388,0.687197,0.637741,0.361756,0.236754,0.353745,0.267383,0.187435,-0.015935,0.459545,0.586312,0.598425,0.543194,0.257394,0.580702,0.715580,0.624350,0.785734,0.075451,0.588862,0.602955,0.674334,0.662148,-0.228776
2002,NaN,-0.096087,0.882475,1.0,0.853660,0.891242,0.796129,0.712807,0.834487,-0.046094,0.678373,0.712362,-0.134412,0.654283,0.818104,0.785548,0.843267,0.727707,0.213262,0.837446,0.913858,0.840019,0.892687,-0.138243,0.857341,0.890370,0.933641,0.932537,0.010598
2003,NaN,-0.278662,0.885719,1.0,0.799561,0.683537,0.716389,0.580380,0.589618,0.031564,0.315628,0.438472,0.188453,0.141965,0.563701,0.624177,0.662481,0.600315,0.198842,0.645694,0.757947,0.663414,0.683051,-0.021223,0.686105,0.673970,0.687997,0.673206,-0.089053
2004,NaN,-0.354269,0.913394,1.0,0.909045,0.708367,0.510095,0.565641,0.571947,0.347055,0.585564,0.090013,-0.239223,0.293136,0.474106,0.567886,0.596805,0.378025,0.289849,0.580602,0.690395,0.622659,0.708948,0.189992,0.593816,0.705393,0.732490,0.734674,0.168267
2005,NaN,-0.486403,0.919206,1.0,0.914022,0.710363,0.578017,0.586368,0.380630,0.507568,0.413137,0.360370,-0.045271,0.285837,0.506668,0.576796,0.602141,0.493503,0.134546,0.590811,0.659684,0.625452,0.710222,0.299609,0.606345,0.644203,0.658037,0.654146,0.122982
2006,NaN,-0.427141,0.900086,1.0,0.851657,0.660211,0.576698,0.622706,0.310064,0.509425,0.343468,0.120617,-0.159496,0.261736,0.313384,0.392930,0.407508,0.250593,0.116703,0.393056,0.596873,0.685087,0.657633,0.256393,0.406372,0.558607,0.597654,0.595496,0.054507
2007,NaN,-0.360005,0.916382,1.0,0.867005,0.585380,0.678049,0.647378,0.321232,0.428338,0.212421,0.038296,0.006229,0.023523,0.149751,0.487308,0.415430,0.444131,-0.023886,0.376609,0.512326,0.665911,0.591573,0.179878,0.366410,0.445148,0.534586,0.529137,-0.220640


In [ ]:
correlation_years.columns

Index(['Year', 'Age', 'Years', 'Guarantee', 'Future WAR', 'past WAR', 'HR',
       'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS',
       'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+', 'WPA',
       'RE24', 'REW', 'Clutch'],
      dtype='object')

In [ ]:
#Generating correlation plots
variable = 'wRC+' #@param ['Year', 'Age', 'Years', 'Guarantee', 'Future WAR', 'Unnamed: 13','past WAR', 'HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP','SLG', 'OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd','wRC+', 'WPA', 'RE24', 'REW', 'Clutch']

px.bar(correlation_years,
       x=correlation_years.index,
       y=variable)

## Regression Model Development

In [ ]:
#@title Scatter Plot

y = 'Guarantee' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']
x = 'HR' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']

# Scatter plots

px.scatter(hitters,
       x= x,
       y= y,
       color='Year',
       hover_data=['Player', 'Guarantee', 'Position']
       )

### Developing OLS Regression Model for Contract Amount

In [ ]:
# Spliting data into X and Y
#Note: This analysis can be done with Future WAR as the dependent variable instead of Guarantee

X = hitters.drop(columns=['Guarantee'])
y = hitters['Guarantee']

# Dropping non-relevant varibales

variables_drop = ['Year', 'Years', 'Future WAR', 'Term', 'Player', 'Position', 'New Club', 'Player Agent', 'Club Owner', 'Old Club', 'Baseball Ops head / club GM']

X.drop(columns=variables_drop, inplace=True)

In [ ]:
X

,Age,past WAR,HR,RBI,IBB,SB,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,RAR,Spd,wRC+,WPA,RE24,REW,Clutch
0,31,7.3,28,116,5,27,0.303,0.088,0.100,0.88,0.365,0.452,0.817,0.149,0.316,0.358,24.6,164,74.8,4.5,115,-0.44,8.42,0.92,-1.62
1,34,10.2,81,231,15,16,0.275,0.104,0.143,0.73,0.355,0.526,0.880,0.251,0.268,0.376,51.4,224,103.3,3.4,127,4.45,47.41,4.57,0.28
2,31,7.6,27,117,20,1,0.263,0.126,0.148,0.86,0.355,0.433,0.788,0.171,0.284,0.341,6.9,117,77.0,3.0,104,3.06,11.46,1.05,3.00
3,34,3.3,53,158,1,18,0.291,0.099,0.175,0.56,0.364,0.530,0.894,0.239,0.310,0.385,46.6,177,33.7,5.1,114,1.56,33.07,3.30,-1.83
4,34,0.8,42,159,2,27,0.254,0.068,0.151,0.45,0.307,0.436,0.743,0.182,0.271,0.323,-10.0,149,8.8,6.3,97,0.31,7.01,0.54,-0.21
5,32,7.7,25,101,2,23,0.299,0.064,0.112,0.57,0.356,0.480,0.836,0.181,0.311,0.364,21.0,117,78.2,6.2,120,-0.09,9.82,0.94,-2.69
6,34,4.2,40,180,23,10,0.282,0.080,0.115,0.70,0.338,0.458,0.796,0.176,0.287,0.339,7.4,159,42.6,3.5,103,-0.22,27.33,2.64,-2.24
7,35,6.0,35,153,16,1,0.314,0.111,0.146,0.76,0.390,0.503,0.893,0.189,0.342,0.385,48.1,182,61.4,2.4,126,1.80,33.23,2.99,-1.16
8,30,9.9,47,197,16,30,0.333,0.126,0.138,0.91,0.415,0.560,0.974,0.226,0.356,0.416,82.5,227,102.0,5.7,153,6.27,76.72,7.32,-0.83
9,33,0.4,6,31,0,1,0.235,0.126,0.264,0.48,0.331,0.394,0.724,0.158,0.309,0.324,-1.6,30,4.4,3.1,73,-0.75,-3.79,-0.41,0.25


In [ ]:
#Testing for multicollinearity by analyzig VIF values. 
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["variable"] = X.columns 
  
# calculating VIF for each variable 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))]

vif_data

,variable,VIF
0,Age,1.387554e+02
1,past WAR,2.712839e+03
2,HR,7.657346e+01
3,RBI,1.222690e+02
4,IBB,9.417963e+00
5,SB,5.965892e+00
6,AVG,3.469202e+05
7,BB%,2.741234e+02
8,K%,2.925931e+02
9,BB/K,3.964468e+01


In [ ]:
# Dropping variables that show multicollinearity and seem to be insignificant

variables_drop = ['Clutch', 'REW','RE24','wRC+','Spd','wRAA','wOBA','BABIP','ISO','OPS','BB/K','K%','BB%','AVG','SB','RBI','past WAR','Age']

X.drop(columns=variables_drop, inplace=True)

In [ ]:
# One hot encoding categorical variables using get_dummies

X_encoded = pd.get_dummies(X, drop_first=True)
# X_encoded = X.copy()

# Filling missing values with median values

for column in X_encoded.columns:
  median = X_encoded[column].median()
  X_encoded[column].fillna(median, inplace=True)

# Creating a train and test set

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(312, 24)
(78, 24)
(312,)
(78,)


In [ ]:
# Normalizing the data

for column in X_train.columns:
  X_train[column] = (X_train[column] - X_train[column].mean()) / X_train[column].std()

In [ ]:
X_train

,Age,past WAR,HR,RBI,IBB,SB,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,RAR,Spd,wRC+,WPA,RE24,REW
459,-1.133669,-0.857993,-0.952605,-0.992416,-0.432156,-0.440494,-1.003209,-1.110471,-0.386373,-0.782311,-1.318387,-1.010028,-1.226985,-0.737564,-0.956534,-1.402185,-0.821280,-1.257982,-0.848920,1.138661,-1.267449,-0.868980,-0.766091,-0.777238
56,-0.514763,-0.051956,1.073621,0.327743,0.376840,-0.543210,-1.247836,0.757275,1.691687,-0.473074,-0.297775,0.948940,0.595081,1.594337,-1.572021,0.414288,-0.015364,-0.103285,0.013136,-1.047568,0.371743,0.312987,0.090190,0.079300
8,-1.133669,2.037769,1.024201,1.685621,1.320668,0.843457,2.503121,1.118775,-0.282470,0.825723,2.223738,1.847415,2.166203,1.057496,2.157105,2.288427,2.618690,1.992934,2.044916,1.138661,2.321053,1.822916,2.108853,2.064194
434,-0.514763,-0.798287,-1.298546,-1.935387,-0.836654,-0.697284,1.035355,-1.803345,-0.407153,-1.308015,-0.087649,-0.877467,-0.681456,-1.425391,1.758848,-0.537198,-0.273536,-1.897507,-0.808284,-1.897768,-0.292794,-0.310468,-0.305867,-0.306300
134,0.413596,-0.529608,-0.952605,-0.898119,-0.701821,1.305679,0.179158,-1.291221,-0.864326,-0.689540,-0.838099,-1.172048,-1.183342,-1.408615,0.274439,-1.200354,-1.239938,-0.405283,-0.520932,1.685218,-1.267449,-0.976136,-1.170131,-1.175481
540,0.413596,0.246576,-0.606664,-0.200321,-0.432156,-0.080988,0.749956,-0.206723,-0.490276,-0.009218,0.422657,-0.303032,-0.070464,-0.636907,0.998541,0.039460,-0.015364,0.109890,0.239534,0.713561,0.061626,0.183100,0.425457,0.414330
445,0.723049,-0.559461,-0.606664,-0.068305,-0.162491,-0.645926,-0.554725,1.179025,0.237045,0.454638,0.662802,-0.715447,-0.288675,-0.586578,0.129619,-0.047038,-0.053741,-0.014462,-0.544152,-0.622468,0.150231,0.946183,0.679364,0.698789
480,-0.824216,-1.216232,-0.606664,-0.275759,-0.297323,0.586667,-1.288608,-0.417597,-0.490276,-0.163836,-1.018207,-1.319339,-1.347001,-0.955656,-1.173765,-1.344519,-1.400423,-0.458576,-1.217543,0.956475,-1.533264,-1.346312,-1.196228,-1.150196
399,-0.824216,0.545108,0.727680,0.893525,-0.027658,-0.645926,0.219930,-0.779097,0.195484,-0.782311,-0.327793,0.315590,0.115016,0.252236,-0.051407,0.010628,-0.172361,0.287536,0.573327,-1.169025,-0.071282,0.059708,0.190586,0.240494
63,1.032502,-0.708727,-0.606664,-0.332337,0.242007,-0.337778,-0.391639,-1.080346,0.070801,-0.936930,-1.198315,-0.759634,-1.008773,-0.704012,-0.087612,-1.258020,-1.180628,-0.725045,-0.706695,-0.258096,-1.267449,-1.252144,-1.133595,-1.203927


In [ ]:
# Applying constant and creating the model

X_train_sm = sm.add_constant(X_train)

model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              Guarantee   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.684
Method:                 Least Squares   F-statistic:                     29.06
Date:                Sat, 08 May 2021   Prob (F-statistic):           7.45e-63
Time:                        02:34:28   Log-Likelihood:                -5573.8
No. Observations:                 312   AIC:                         1.120e+04
Df Residuals:                     287   BIC:                         1.129e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.205e+07   8.19e+05     14.709      0.0

### Developing Machine Learning Model with Pycaret for Contract Amount

In [ ]:
hitters.columns

Index(['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years',
       'Guarantee', 'Term', 'Player Agent', 'Club Owner',
       'Baseball Ops head / club GM', 'Future WAR', 'past WAR', 'HR', 'RBI',
       'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO',
       'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+', 'WPA', 'RE24',
       'REW', 'Clutch'],
      dtype='object')

In [ ]:
# Preparing the training data

setup(hitters, 
      target='Guarantee',
      # numeric_features = ['Years', 'HR', 'HBP', 'WP', 'W'],
      numeric_features = ['Age'],
      ignore_features = ['Player',
                         'Future WAR',
                         'Years',
                         'Term'],
      normalize = True, 
      normalize_method = 'zscore',
      )

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,02:34:30
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
Year,Categorical
Position,Categorical
Age,Numeric
Old Club,Categorical
New Club,Categorical
Player Agent,Categorical
Club Owner,Categorical
Baseball Ops head / club GM,Categorical
past WAR,Numeric
HR,Numeric


In [ ]:
#Comparing performance of different machine learning models
compare_models()

In [ ]:
# Putting results into a dataframe

dataframe_model_comparison = pull()

dataframe_model_comparison.to_excel('model_comparison.xlsx')

dataframe_model_comparison

In [ ]:
# Choosing Random Forest Model
model = create_model('rf')

In [ ]:
#Generating feature importance plot
plot_model(model, plot = 'variable')

In [ ]:
# Putting results of model fitting into a dataframe

model_fitting_df = pull()

model_fitting_df.to_excel('model_fitting.xlsx')

model_fitting_df

In [ ]:
plot_model(model, plot = 'feature')

In [ ]:
# Generating importance plot from 1999-2004

years = (1999, 2004)

hitters_ml = hitters[hitters['Year'] >= start]
hitters_ml = hitters_ml[hitters_ml['Year'] <= end]

setup(hitters_ml, 
    target='Guarantee',
    # numeric_features = ['Years', 'HR', 'HBP', 'WP', 'W'],
    numeric_features = ['Age'],
    ignore_features = ['Player',
                        'Future WAR',
                        'Years',
                        'Unnamed: 13',
                        'Term'],
    normalize = True, 
    normalize_method = 'zscore',
    silent = True,
    verbose = False)

model = create_model('rf',
                      verbose = False)

print("Importance plot for years 1999, 2004")

plot_model(model, plot = 'feature')

In [ ]:
#Generating importance plot from 2005-2010
years = (2005, 2010)

hitters_ml = hitters[hitters['Year'] >= start]
hitters_ml = hitters_ml[hitters_ml['Year'] <= end]

setup(hitters_ml, 
    target='Guarantee',
    # numeric_features = ['Years', 'HR', 'HBP', 'WP', 'W'],
    numeric_features = ['Age'],
    ignore_features = ['Player',
                        'Future WAR',
                        'Years',
                        'Unnamed: 13',
                        'Term'],
    normalize = True, 
    normalize_method = 'zscore',
    silent = True,
    verbose = False)

model = create_model('rf',
                      verbose = False)

print("Importance plot for years 2005, 2010")

plot_model(model, plot = 'feature')

In [ ]:
# Generating importnace plot for the full time period
years = (1999, 2019)

hitters_ml = hitters[hitters['Year'] >= start]
hitters_ml = hitters_ml[hitters_ml['Year'] <= end]

setup(hitters_ml, 
    target='Guarantee',
    # numeric_features = ['Years', 'HR', 'HBP', 'WP', 'W'],
    numeric_features = ['Age'],
    ignore_features = ['Player',
                        'Future WAR',
                        'Years',
                        'Unnamed: 13',
                        'Term'],
    normalize = True, 
    normalize_method = 'zscore',
    silent = True,
    verbose = False)

model = create_model('rf',
                      verbose = False)

print("Importance plot for years 1999, 2019")

plot_model(model, plot = 'feature')

### Developing Machine Learning Model with Pycaret for Future WAR

In [ ]:
# Preparing the training data


setup(hitters, 
      target='Future WAR',
      # numeric_features = ['Years', 'HR', 'HBP', 'WP', 'W'],
      numeric_features = ['Age'],
      ignore_features = ['Player',
                         'Years',
                         'Unnamed: 13',
                         'Term'],
      normalize = True, 
      normalize_method = 'zscore',
      )

In [ ]:
#Comparing models
compare_models()